<a href="https://colab.research.google.com/github/pradeep058888/Project3/blob/main/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

train_path = "/content/drive/MyDrive/Project3/train.csv"
test_path = "/content/drive/MyDrive/Project3/test.csv"

# Train check
print("Train file preview:")
train_check = pd.read_csv(train_path, nrows=10)
print(train_check)
print("Train shape:", pd.read_csv(train_path).shape)


Train file preview:
   age           job   marital          education  default housing loan  \
0   49   blue-collar   married           basic.9y  unknown      no   no   
1   37  entrepreneur   married  university.degree       no      no   no   
2   78       retired   married           basic.4y       no      no   no   
3   36        admin.   married  university.degree       no     yes   no   
4   59       retired  divorced  university.degree       no      no   no   
5   29        admin.    single  university.degree       no      no   no   
6   26       student    single           basic.9y       no      no   no   
7   30   blue-collar   married           basic.4y       no     yes   no   
8   50   blue-collar   married           basic.4y  unknown      no   no   
9   33        admin.    single        high.school       no     yes   no   

     contact month day_of_week  duration  campaign  pdays  previous  \
0   cellular   nov         wed       227         4    999         0   
1  telephone

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

train_path = "/content/drive/MyDrive/Project3/train.csv"
test_path = "/content/drive/MyDrive/Project3/test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train shape: (32950, 16)
Test shape: (8238, 13)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,49,blue-collar,married,basic.9y,unknown,no,no,cellular,nov,wed,227,4,999,0,nonexistent,no
1,37,entrepreneur,married,university.degree,no,no,no,telephone,nov,wed,202,2,999,1,failure,no
2,78,retired,married,basic.4y,no,no,no,cellular,jul,mon,1148,1,999,0,nonexistent,yes
3,36,admin.,married,university.degree,no,yes,no,telephone,may,mon,120,2,999,0,nonexistent,no
4,59,retired,divorced,university.degree,no,no,no,cellular,jun,tue,368,2,999,0,nonexistent,no


In [ ]:
# Encode target
train['y'] = train['y'].map({'yes':1, 'no':0})

# Separate features and target
y = train['y']
X = train.drop(columns=['y'])

# Identify categorical columns
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

# One-hot encode
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=True)
test_encoded = pd.get_dummies(test, columns=cat_cols, drop_first=True)

# Align train and test
X_encoded, test_encoded = X_encoded.align(test_encoded, join='left', axis=1, fill_value=0)

print("Final training feature shape:", X_encoded.shape)
print("Final test feature shape:", test_encoded.shape)


Final training feature shape: (32950, 48)
Final test feature shape: (8238, 48)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:", X_val.shape, "y_val:", y_val.shape)


X_train: (26360, 48) y_train: (26360,)
X_val: (6590, 48) y_val: (6590,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate_model(model, X_val, y_val):
    preds = model.predict(X_val)
    proba = model.predict_proba(X_val)[:,1]
    print("Accuracy:", accuracy_score(y_val, preds))
    print("Precision:", precision_score(y_val, preds))
    print("Recall:", recall_score(y_val, preds))
    print("F1 Score:", f1_score(y_val, preds))
    print("ROC-AUC:", roc_auc_score(y_val, proba))
    print("-"*40)

# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
print("Logistic Regression Performance:")
evaluate_model(log_reg, X_val, y_val)

# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
print("Random Forest Performance:")
evaluate_model(rf, X_val, y_val)

# XGBoost
xgb = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6,
                    random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
print("XGBoost Performance:")
evaluate_model(xgb, X_val, y_val)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Performance:
Accuracy: 0.9033383915022761
Precision: 0.6335877862595419
Recall: 0.33557951482479786
F1 Score: 0.43876651982378856
ROC-AUC: 0.9090561520790853
----------------------------------------
Random Forest Performance:
Accuracy: 0.9039453717754173
Precision: 0.6306954436450839
Recall: 0.35444743935309975
F1 Score: 0.453839516824849
ROC-AUC: 0.9194825747323939
----------------------------------------


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [07:49:34] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost Performance:
Accuracy: 0.9081942336874052
Precision: 0.6261510128913443
Recall: 0.4582210242587601
F1 Score: 0.5291828793774319
ROC-AUC: 0.9287124678743811
----------------------------------------


In [ ]:
# Use best model (XGBoost)
test_preds = xgb.predict(test_encoded)

submission = pd.DataFrame({
    'ID': range(1, len(test_preds)+1),
    'y_pred': test_preds
})

submission.to_csv("/content/drive/MyDrive/Project3/submission.csv", index=False)
print("Submission file saved!")


Submission file saved!
